# 교차 검증과 그리드 서치

테스트 세트에만 맞춰서 만들 경우.. 실제로는 점수가 내려갈 수도 있다.
그래서 검증 세트를 만든다.

말 그대로 검증을 하기 위한 값임.

순서는 .. 훈련 -> 검증 -> 테스트

테스트 점수가 좀 더 낮을수도 있다.


<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/hg-mldl2/blob/main/05-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## 검증 세트

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

In [6]:
# test_input 의 비율을 20%로 만든다.

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [7]:
# train_input 의 20%로 val_input 생성

sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [8]:
# 훈련 4157개, 검증 1040개
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [10]:
#훈련 99, 검증 86, 테스트 85점

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
print(dt.score(test_input, test_target))

0.9971133028626413
0.864423076923077
0.8569230769230769


## 교차 검증

In [12]:
# 교차검증을 통하여 더 좋은 점수를 얻어보자.

# 보통은 5-폴드 교차 검증.
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01475191, 0.01474595, 0.01579928, 0.01500559, 0.01478767]), 'score_time': array([0.00394988, 0.0041368 , 0.00391769, 0.00399351, 0.00448608]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [14]:
# 앞에서 한것보다 더 안정된 점수이다.

import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [16]:
# K Fold : 타깃 클래스를 골고루 나누기 위함.
# 앞에서는 train_test_split 을 통하여 섞었으나, cross_validate는 훈련 세트를 섞지 않음.

from sklearn.model_selection import StratifiedKFold

# ex) cv = 5 or 10 ..
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [17]:
# 값을 섞어서 10-폴드 교차검증을 수행한다.

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

매개변수가 많아진다면 간단한 for문을 이용하여서 최적의 값을 구하기가 더더욱 어렵다.

그러므로 import하여 사용하자.. (GridSearchCV)

In [31]:
from sklearn.model_selection import GridSearchCV
# min_impurity_decrease (최소 불순도 ?)
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [32]:
# 결정 트리 생성 DecisionTreeClassifier
# n_jobs : cpu 코어에 교차 검증을 수행할 수 있게 함. -1로 하면 모든 코어를 사용

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [33]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [34]:
# 교차 검증으로 찾은 가장 좋은 파라미터 : best_estimator_

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [37]:
# 생각보다 많은 내용이 나오진 않는다.
print(gs.best_estimator_)

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)


In [38]:
# 최적의 매개변수를 확인.
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [39]:
# 교차검증의 평균 점수. (5 폴드)
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [40]:
print(gs.cv_results_['params'][gs.best_index_])

{'min_impurity_decrease': 0.0001}


In [41]:
# min_impurity_decrease의 값이 0.0001 부터 0.001 까지 0.0001씩 증가를 하고
# max_depth 은 5 부터 20 까지 1씩 증가.
# min_samples_split 은 2부터 100까지 10씩 증가.

# ==> 상당히 많은 모델을 만들었다.
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [42]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [43]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [44]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

우리가 아까 앞에서 정의한 값의 범위(range)는 사용자 임의의값이였음.

기준도 없었다.

그러면서 너무 많은 조건도 있어서 오래 걸릴 수 있다..

임의로 매개변수 값을 샘플링 하여 객체를 전달하자.

In [45]:
from scipy.stats import uniform, randint

In [50]:
# 정수로 뽑는다.
rgen = randint(0, 10)
rgen.rvs(10)

array([4, 1, 5, 7, 9, 9, 4, 6, 7, 6])

In [58]:
# 0 ~ 9 까지 를 1000번 뽑는다.
# 확률상 서로 비슷한 확률로 추첨이 됨.

np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 89, 118,  91,  99, 115,  94,  99, 105,  93,  97]))

In [59]:
# 실수로 뽑는다.

ugen = uniform(0, 1)
ugen.rvs(10)

array([0.18770987, 0.08596749, 0.08882061, 0.90269266, 0.7924398 ,
       0.50001543, 0.49362936, 0.73736242, 0.9786341 , 0.39201188])

In [60]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [61]:
# n_iter :  몇번 할것인가?

from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fe3768ae550>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fe376736a10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fe376734050>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fe3768b0310>},
                   random_state=42)

In [62]:
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


In [63]:
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [64]:
dt = rs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [65]:
# splitter의 값이 random 임.. 점수가 내려갔다. 왜 ?

gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fe3768ae550>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fe376736a10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fe376734050>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fe3768b0310>},
                   random_state=42)

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
